<a href="https://colab.research.google.com/github/cagoodri2/cagoodri2.github.io/blob/master/Sample_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Corissa Goodrich

Project: Hybrid RNN for LSST Dataset

The following code is not for reuse or replication by any party other than the original author.

In [ ]:
#Load Libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn
import imblearn
from imblearn.over_sampling import RandomOverSampler
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#Accuracy & Loss Plots
def accuracy_plot(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs_range = range(len(acc))
    plt.figure(figsize=(6,4))
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.show()

def loss_plot(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(loss))
    plt.figure(figsize=(6,4))
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [ ]:
### SAMPLE CODE FOR MODEL BUILD ###
def build_model(hp):
    #define shape
    input_shape = (6, 36)

    #normalize
    normalizer = tf.keras.layers.Normalization()
    normalizer.adapt(X_train)

    #input with defined shape
    inputs = tf.keras.layers.Input(shape=input_shape)

    #normalize inputs
    x = normalizer(inputs)

    #RNN layers
    x = tf.keras.layers.SimpleRNN(units=hp.Int('units_1', min_value=32, max_value=128, step=32),
                                  activation='relu',
                                  return_sequences=True)(x)
    x = tf.keras.layers.Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1))(x)
    x = tf.keras.layers.Conv1D(filters=hp.Int('filters_1', min_value=16, max_value=64, step=16),
                               kernel_size=hp.Int('kernel_size_1', min_value=2, max_value=5),
                               activation='relu',
                               padding='same')(x)
    x = tf.keras.layers.SimpleRNN(units=hp.Int('units_2', min_value=32, max_value=128, step=32),
                                  activation='relu',
                                  return_sequences=True)(x)
    x = tf.keras.layers.Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1))(x)
    x = tf.keras.layers.Dense(units=hp.Int('dense_units', min_value=32, max_value=128, step=32),
                              activation='relu')(x)
    x = tf.keras.layers.Flatten()(x)

    #ouputs
    outputs = tf.keras.layers.Dense(units=11, activation='softmax')(x)

    #model compile
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    opt = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
### SAMPLE CODE FOR MODEL TUNING & TESTING
stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner = RandomSearch(build_model, objective = 'val_accuracy', max_trials = 10, directory = 'hybrid_33', project_name= 'hybrid_33')

In [ ]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), callbacks=[stopping])

In [ ]:
best_params = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters
print(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)

In [ ]:
accuracy_plot(history)

In [ ]:
loss_plot(history)

In [ ]:
predictions = model.predict(X_test)